In [1]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import datasets

In [2]:
t_pipeline = transforms.Compose([ 
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
   transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), 
  transforms.Resize((224, 224)),          
      transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5],) 
])

In [3]:
data_dir_train =  r'C:\Users\POORNIMA\Desktop\POORNIMA\DataScienceProjects\Final Project\brain_tumor_dataset (1)\Training' 
data_dir_test =  r'C:\Users\POORNIMA\Desktop\POORNIMA\DataScienceProjects\Final Project\brain_tumor_dataset (1)\Testing' 
train_dataset = ImageFolder(root=data_dir_train, transform = t_pipeline)
test_dataset = ImageFolder(root=data_dir_test, transform = t_pipeline)


trainloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
testloader = DataLoader(test_dataset, batch_size = 64, shuffle = False)


In [4]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 5712
    Root location: C:\Users\POORNIMA\Desktop\POORNIMA\DataScienceProjects\Final Project\brain_tumor_dataset (1)\Training
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [5]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 1311
    Root location: C:\Users\POORNIMA\Desktop\POORNIMA\DataScienceProjects\Final Project\brain_tumor_dataset (1)\Testing
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [6]:
print("Class labels:", test_dataset.classes)

Class labels: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [7]:
print("Class labels:", train_dataset.classes)

Class labels: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [8]:
for i, (images, labels) in enumerate(trainloader):
    print(f"Index: {i}, Image Shape: {images.shape}, Label: {labels}")
    break 

Index: 0, Image Shape: torch.Size([64, 3, 224, 224]), Label: tensor([1, 2, 1, 3, 2, 2, 2, 0, 1, 0, 1, 3, 2, 3, 2, 3, 1, 1, 1, 2, 0, 3, 0, 0,
        2, 1, 0, 2, 3, 1, 3, 2, 1, 1, 3, 0, 2, 0, 1, 0, 1, 0, 3, 1, 3, 3, 1, 2,
        2, 2, 3, 2, 2, 0, 3, 0, 2, 3, 3, 1, 3, 3, 3, 0])


In [9]:
for i, (images, labels) in enumerate(train_dataset):
    print(f"Index: {i}, Image Shape: {images.shape}, Label: {labels}")
    if i==10:
        break 

Index: 0, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 1, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 2, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 3, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 4, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 5, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 6, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 7, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 8, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 9, Image Shape: torch.Size([3, 224, 224]), Label: 0
Index: 10, Image Shape: torch.Size([3, 224, 224]), Label: 0


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Max pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # 128 channels, 28x28 after pooling
        self.fc2 = nn.Linear(512, 4)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        # Flatten the output for fully connected layer
        x = x.view(-1, 128 * 28 * 28)
        
        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [13]:
model = CNN()
loss_obj = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1):
    losses = []
    for data in trainloader:
        images, labels = data
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_obj(outputs, labels)

        losses.append(loss.item())

        loss.backward()
        optimizer.step()

       
        print(f"Epoch {epoch+1}, Batch Loss: {loss.item()}")

   
    avg_loss = sum(losses) / len(losses)
    print(f"Epoch {epoch+1} Average Loss: {avg_loss}")


Epoch 1, Batch Loss: 1.3845769166946411
Epoch 1, Batch Loss: 1256.358642578125
Epoch 1, Batch Loss: 18.275327682495117
Epoch 1, Batch Loss: 1.974390983581543
Epoch 1, Batch Loss: 4.436795711517334
Epoch 1, Batch Loss: 1.471383810043335
Epoch 1, Batch Loss: 2.493215322494507
Epoch 1, Batch Loss: 1.2706146240234375
Epoch 1, Batch Loss: 1.4790610074996948
Epoch 1, Batch Loss: 1.402233600616455
Epoch 1, Batch Loss: 1.354838252067566
Epoch 1, Batch Loss: 1.3615484237670898
Epoch 1, Batch Loss: 1.3350474834442139
Epoch 1, Batch Loss: 1.3375974893569946
Epoch 1, Batch Loss: 1.3462576866149902
Epoch 1, Batch Loss: 1.2608412504196167
Epoch 1, Batch Loss: 1.2946237325668335
Epoch 1, Batch Loss: 1.1973652839660645
Epoch 1, Batch Loss: 1.1454219818115234
Epoch 1, Batch Loss: 1.2711600065231323
Epoch 1, Batch Loss: 1.2793382406234741
Epoch 1, Batch Loss: 1.3292789459228516
Epoch 1, Batch Loss: 1.0588607788085938
Epoch 1, Batch Loss: 1.171890377998352
Epoch 1, Batch Loss: 1.1307134628295898
Epoch 1,